<a href="https://colab.research.google.com/github/matan99/Double-Pendulum/blob/main/model/data-extraction/model-validation/cosmos/CosmosDataExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Variable Parameters as per user's preferences

In [ ]:
N_BLOCKS_TO_GET = 30
VALIDATOR_NAME = ["Coinbase", "Twinstake", "Figment"]
VALIDATOR_ADDRESS = ["cosmosvaloper1c4k24jzduc365kywrsvf5ujz4ya6mwympnc4en", "cosmosvaloper1svwt2mr4x2mx0hcmty0mxsa4rmlfau4lwx2l69", "cosmosvaloper1hjct6q7npsspsg3dgvzk3sdf89spmlpfdn6m9d"]
base_dir_path = '/content/drive/Shareddrives/Model/Data Extraction/COSMOS/'
from datetime import date
datafile_name = "multiple-"+str(date.today())
print(f"Saving as file {datafile_name}")

Saving as file multiple-2022-11-23


### Importing libraries and functions

In [ ]:
import time
from tqdm import tqdm
import threading
import calendar
import json
from collections import defaultdict
import os
import pandas as pd
from threading import Thread
from time import sleep
from datetime import datetime
import datetime as datetime_alias
import requests
from google.colab import drive
drive.mount('drive')

Mounted at drive


### Defining functions

In [ ]:
MAX_TXNS_PER_BLOCK = 50
RPC_URL = 'https://api.cosmos.network'
COSMOSCAN_API = 'https://api.cosmoscan.net'
RPC_URL_PRIVATE="https://cosmos-mainnet-rpc.allthatnode.com:1317/pBuvbXvskIvU2qNrs3yPHOQtFsZwM7Wj"
headers = {'accept': 'application/json'}
CMC_headers = {
    'X-CMC_PRO_API_KEY': '',
    'Accept': 'application/json',
}
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
def get_inflation():    
    response = requests.get(RPC_URL+'/cosmos/mint/v1beta1/inflation', headers=headers).json()
    return response['inflation']

# param - block_num can be supplied to get the data for a specific block. if the paramter is not specified the function returns the data fro the latest block.
def get_timestamp(block_number = -1):    
    if block_number == -1:
        response = requests.get(RPC_URL+'/cosmos/base/tendermint/v1beta1/blocks/latest', headers=headers).json()
    else:
        response = requests.get(RPC_URL+'/cosmos/base/tendermint/v1beta1/blocks/'+str(block_number), headers=headers).json()
    block_timestamp = response['block']['header']['time']
    return block_timestamp

def get_supply_bonded_ratio():
    params = {
        'by': 'hour',
    }
    bond_ratio = requests.get('https://api.cosmoscan.net/bonded-ratio/agg', params=params, headers=headers).json()[-1]['value']
    return bond_ratio

# returns total validator set size (425)
def get_n_validators():
    response = requests.get(COSMOSCAN_API+'/validators', headers=headers).json()     
    return len(response)
    
def get_n_active_validators():
    validator_set = requests.get(RPC_URL_3+'/validatorsets/latest', headers=headers).json()['result']['total']
    return int(validator_set)

def get_validator_stake(validator_addr):    
    stake_val = requests.get(COSMOSCAN_API+'/validator/'+validator_addr, headers=headers).json()['power']
    return stake_val

# 2 API requests are made inside this function
def get_precommit_ratio(num_signatures):
    response = len(requests.get(RPC_URL+'/blocks/'+str(num_signatures), headers=headers).json()['block']['last_commit']['signatures'])
    return response

# returns a dictionary containing: community tax to be paid, miniumum proposer bonus, and maximum proposer bonus
def get_chain_distribution_parameters():
    response = requests.get(RPC_URL+'/distribution/parameters', headers=headers).json()['result']
    result_dict = {}
    result_dict['community_tax'] = response['community_tax']
    result_dict['min_proposer_bonus'] = response['base_proposer_reward']
    result_dict['max_proposer_bonus'] = response['bonus_proposer_reward']
    return result_dict

def get_rewards_current(validator_addr):
    """
    Returns the commission earned by the validator till date
    """
    response = requests.get(RPC_URL+f'/cosmos/distribution/v1beta1/validators/{validator_addr}/commission', headers=headers).json()
    result_dict = {}
    response = response['commission']['commission'][0]['amount']
    return float(response)

def list_to_dict(ls, keys):
    d = {}
    for i in range(len(ls)):
        d[keys[i]] = ls[i]
    return d

def get_total_supply():
    response = requests.get(RPC_URL+'/cosmos/bank/v1beta1/supply/uatom', headers=headers).json()['amount']['amount']
    return response

def get_block_time(bl_num):
    response = requests.get(RPC_URL+'/blocks/'+str(bl_num), headers=headers).json()['block']['header']['time']
    return response

def get_total_fees(_block_num):
    params = {
        'events': 'tx.height='+str(_block_num),
        'pagination.limit': '20',
    }
    response = requests.get('https://api.cosmos.network/cosmos/tx/v1beta1/txs', params=params, headers=headers).json()['tx_responses']
    total_fees = 0
    for i in response:
        total_fees += int(i['tx']['auth_info']['fee']['amount'][0]['amount'])
    return total_fees

def get_validator_commission(validator_addr):
    """
    Returns the commission percentage of the validator
    """
    response = requests.get(RPC_URL+f'/cosmos/staking/v1beta1/validators/{validator_addr}', headers=headers).json()['validator']['commission']['commission_rates']['rate']
    response = float(response)*100
    return response

# param ls_valid requires the list of validators for which the commission and stake is to be returned
def get_ALL_validators_info(ls_valid):
    res_dict = {}
    all_info = requests.get(COSMOSCAN_API+'/validators', headers=headers).json()

    index = 1
    for val_addr in ls_valid:
        for info in all_info:
            if val_addr == info['operator_address']:
                res_dict["v"+str(index)+"_title"] = info['title']
                res_dict["v"+str(index)+"_id"] = val_addr
                res_dict["v"+str(index)+"_comsn"] = info['fee']
                res_dict["v"+str(index)+"_stake"] = info['power']
                index+=1
                break
    return res_dict

# returns the sum of active validators' stake and inactive validators' stake (in ATOM)
def get_active_stake_vs_inactive():
    all_info = requests.get(COSMOSCAN_API+'/validators', headers=headers).json()
    stakes = []
    for i in range(len(all_info)):
        stakes.append(float(all_info[i]['power']))
    stakes = np.array(stakes)
    stakes.sort()
    active_sum = stakes[-175:].sum()
    inactive_sum = stakes[:-175].sum()
    return active_sum, inactive_sum

# desc returns the signing ratio for a block
# param block_signatures - an array of signatures for a block
# param n_act_validators - number of active validators on the network
def get_sign_ratio_from_signatures_array(block_signatures, n_act_validators):
    count=0
    for i in block_signatures:
        if i['block_id_flag'] == 'BLOCK_ID_FLAG_COMMIT':
            count+=1
    return count/n_act_validators

# desc returns the array of signatures from each active validator for a specific block number
# param _block_num - the height of the block for which the signatures are to be returned
def get_signatures(_block_num):
    url = RPC_URL+f'/blocks/{_block_num}'
    block_info = requests.get(url, headers=headers).json()['block']['last_commit']['signatures']
    return block_info

def get_rewards(BLOCK):
    def def_value():
        return 0
    val_rewards = defaultdict(def_value)

    # url = f'https://rpc-cosmoshub.blockapsis.com/block_results?height={BLOCK}'  # Can use this RPC if the next line fails
    url = f'https://rpc.cosmos.network/block_results?height={BLOCK}'
    response = requests.get(url=url, headers=headers)
    begin_block_events = response.json()['result']['begin_block_events']
    print(set([i['type'] for i in begin_block_events]))
    for event in begin_block_events:
        if event['type'] == "rewards": # "proposer_reward" type event intentionally not mentioned because it is duplicated as "reward" as well
            if len(event['attributes']) != 2:
                # Probably skipping event with wrong shape
                pass
            else:
                a0 = event['attributes'][0]
                a1 = event['attributes'][1]
                try:
                    key0 = base64.b64decode(a0['key']).decode("utf-8")
                    value0 = base64.b64decode(a0['value']).decode("utf-8")
                    key1 = base64.b64decode(a1['key']).decode("utf-8")
                    value1 = base64.b64decode(a1['value']).decode("utf-8")
                    if key0 == "amount" and key1 == "validator" and event['type'] == 'rewards':
                        val_rewards[value1] += float(value0[:-5])
                except TypeError:
                    # Probably "argument should be a bytes-like object or ASCII string, not 'NoneType'"
                    # Probably missing key or value
                    pass
    
    return val_rewards


def zulu_time_to_timestamp(zulu):
    year = int(zulu.split("T")[0].split("-")[0])
    month = int(zulu.split("T")[0].split("-")[1])
    day = int(zulu.split("T")[0].split("-")[2])
    hour = int(zulu.split("T")[1].split(":")[0])
    min = int(zulu.split("T")[1].split(":")[1])
    sec = int(zulu[:-1].split("T")[1].split(":")[2]) # remove the Z at the end
    date_time = datetime(year, month, day, hour, min, sec)
    unix = calendar.timegm(date_time.timetuple())
    return unix

def get_recent_withdrawals(delegator_ad, val_ad, low_timestamp, high_timestamp):
    low_timestamp = float(low_timestamp)
    high_timestamp = float(high_timestamp)

    url = f"https://api-cosmos.cosmostation.io/v1/account/new_txs/{delegator_ad}?limit=50&from=0"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:107.0) Gecko/20100101 Firefox/107.0'
    }
    response = requests.get(url, headers = headers).json()

    was_withdrawn = False
    total_withdrawn = 0
    last_withdrawal = -1

    for tx in response:
        time_str = zulu_time_to_timestamp(tx['header']['timestamp'])

        if time_str < low_timestamp or time_str > high_timestamp:
            print(time_str)
            continue

        logs = tx['data']['logs']
        for log in logs:
            events = log['events']
            for event in events:
                if event['type'] == 'withdraw_rewards' and event['attributes'][1]['value'] == val_ad:
                    amt_withdrawn = event['attributes'][0]['value']
                    amt_withdrawn = float(amt_withdrawn[:-5]) # assuming the amount is in the form of "1234uatom"
                    was_withdrawn = True
                    total_withdrawn += amt_withdrawn
                    last_withdrawal = time_str

    return total_withdrawn, last_withdrawal

# convert zulu time to unix timestamp
def time_to_unix(time):
    year = int(time.split('-')[0])
    month = int(time.split('-')[1])
    date = int(time.split('-')[2].split('T')[0])
    hour = int(time.split("T")[1].split(":")[0])
    mins = int(time.split("T")[1].split(":")[1])
    secs = int(time.split("T")[1].split(":")[2].split('.')[0])
    millisecs = int(int(time.split("T")[1].split(":")[2].split('.')[1][:-1])/1000000)
    date_time = datetime(year, month, date, hour, mins, secs)
    unix_timestamp = datetime.timestamp(date_time)*1000
    unix_timestamp+=millisecs
    return unix_timestamp 

# param df - DataFrame containing Data Extraction code
def post_process_data(df, N_validators):
    DEFAULT_BLOCK_TIME = 6.5
    # Read the dataframe and sort by values
    ROWS_APPENDED= 0
    df.sort_values(by=['block_num'], inplace=True)
    
    new_df_ls = []
    prev_timestamp = time_to_unix(df.iloc[0]["timestamp"])
    last_block_num = df.iloc[0]["block_num"]
    print(ROWS_APPENDED)
    for ind, row in df.iterrows():
        if ind == 0:
            row['block_len'] = DEFAULT_BLOCK_TIME# for the first row the block length can be set to 6.5 s
        elif row['block_num'] != last_block_num+1:
            # in case we missed a block in between during dat extraction, replace block length with the average time
            time_diff = float(time_to_unix(row['timestamp']) - prev_timestamp)/1000.0
            row['block_len'] = time_diff/float(row['block_num'] - last_block_num)
        else:
            # Otherwise block lenght is time difference between two consecutive blocks
            row['block_len'] = float(time_to_unix(row['timestamp']) - prev_timestamp)/1000.0
        row['appended_block']=0

        new_df_ls.append(row)
        prev_timestamp = time_to_unix(row['timestamp'])
        last_block_num = row['block_num']
    
        new_row = pd.Series(row).copy(deep=True)

        if ind >= len(df)-1:
            continue # continue if the row is last row of the DataFrame

        diff_from_next_row = int(df.iloc[ind+1]['block_num'] - row['block_num']-1)

        rew_diffs = []
        for i in range(N_validators):
            # Find the commission differences for two consecutive rows for all validators 
            rew_diffs.append(df.iloc[ind+1][f'val_{i}_com_amt'] - row[f'val_{i}_com_amt']) 

        row_block_num = row['block_num']
        row_rewards = []
        for i in range(N_validators):
            # Find the current commissions for two consecutive rows for all validators 
            row_rewards.append(row[f'val_{i}_com_amt']) 

        for i in range(diff_from_next_row):
            new_row = pd.Series(new_row).copy(deep=True)
            new_row['block_num'] = row_block_num + i + 1

            for i in range(N_validators):
                new_row[f'val_{i}_com_amt'] = row_rewards[i] + rew_diffs[i]/(diff_from_next_row+1)*(i+1)

            new_row['appended_block'] = 1
            i+=1
            new_df_ls.append(new_row)  
            
    print("Rows appended: ", ROWS_APPENDED)
    # get the processed DataFrame
    new_df = pd.DataFrame(new_df_ls)
    return new_df

### Loading pre-requisite data

In [ ]:
VALIDATOR_STAKES = []
assert len(VALIDATOR_ADDRESS) == len(VALIDATOR_NAME), "VALIDATOR_ADDRESS and VALIDATOR_NAME must have the same length"
for i in range(len(VALIDATOR_ADDRESS)):
    VALIDATOR_STAKES.append(get_validator_stake(VALIDATOR_ADDRESS[i]))

N_ACTIVE_VALIDATORS = int(requests.get(RPC_URL+'/validatorsets/latest', headers=headers).json()['result']['total'])
CHAIN_DISTRIBUTION_PARAMS = get_chain_distribution_parameters()


### Defining threads for DE

In [ ]:
print("Loaded libraries...")

# CONST VALUES - 9,10,11,12,13,17,21
BLOCKS_PRE_YEAR = 4360000
CONST_ATTRIBUTES = {
        "Block_length_target": (365*24*60*60)/BLOCKS_PRE_YEAR,
        "Goal_Bonded": 0.6666,
        "Inflation_Rate_Change": 0.13,
        "Min_Inflation_Rate": 0.07,
        "Max_Inflation_Rate": 0.20,
        "Min_Signatures": 0.6666,
        "Blocks_per_year": BLOCKS_PRE_YEAR,
        "n_validators": N_ACTIVE_VALIDATORS
    }

for i in range(len(VALIDATOR_ADDRESS)):
    CONST_ATTRIBUTES[f'val_{i}_name'] = VALIDATOR_NAME[i]
    CONST_ATTRIBUTES[f'val_{i}_address'] = VALIDATOR_ADDRESS[i]
    CONST_ATTRIBUTES[f'val_{i}_commission_pct'] = get_validator_commission(VALIDATOR_ADDRESS[i])


print(CONST_ATTRIBUTES)

# No API calls to get data
def MyThread0(res, _latest_block):
    t = time.time()
    res['block_num'] = _latest_block
    for key in CONST_ATTRIBUTES.keys():
        res[key] = CONST_ATTRIBUTES[key]
    print(time.time()-t, "s for thread 0")

# 1 - INFLATION RATE
def MyThread1(res, key):
    t = time.time()
    res[key] = get_inflation() 
    print(time.time()-t, "s for thread 1")

# 0 - PERCENTAGE ATOM STAKED
def MyThread2(res, key):
    t = time.time()
    res[key] = get_supply_bonded_ratio()
    print(time.time()-t, "s for thread 2")

# 2, 20 - Total_Fees_Per_Block and txFees
def MyThread3(res, key, _latest_block):
    t = time.time()
    # res[key] = get_total_fees(_latest_block)
    res[key] = 0
    print(time.time()-t, "s for thread 3")

# 4 - block timestamp and proposer address
def MyThread4(res, key, _data):
    t = time.time()
    res[key] = _data
    print(time.time()-t, "s for thread 4")

# 5 - PRECOMMITS RATIO
def MyThread5(res, key, _block_signatures):
    t = time.time()
    res[key] = get_sign_ratio_from_signatures_array(_block_signatures, N_ACTIVE_VALIDATORS)
    print(time.time()-t, "s for thread 5")

# 6 - ATOM STAKED BY VALIDATOR
def MyThread6(res, key):
    t = time.time()
    for i in range(len(VALIDATOR_ADDRESS)):
        res[f'val_{i}_stake'] = VALIDATOR_STAKES[i]
    print(time.time()-t, "s for thread 6")

# 7 - TOTAL SUPPLY 
def MyThread7(res, key):
    t = time.time()
    res[key] = get_total_supply()
    print(time.time()-t, "s for thread7")

# 8 - N ACTIVE VALIDATORS
def MyThread8(res, key):
    t = time.time()
    res[key] = N_ACTIVE_VALIDATORS
    print(time.time()-t, "s for thread 8")

# 14, 15, 16 - min proposer bonus, max proposer bonus, community tax
def MyThread9(res):
    t = time.time()
    for key in CHAIN_DISTRIBUTION_PARAMS.keys():
        res[key] = CHAIN_DISTRIBUTION_PARAMS[key]
    print(time.time()-t, "s for thread 9")

# 18 - Rewards
def MyThread10(res, validator_addr, validator_index):
    t = time.time()
    res[f'val_{validator_index}_com_amt'] = get_rewards_current(validator_addr)
    print(time.time()-t, "s for thread 10")

def get_all_block_data(LATEST_BLOCK, _block_signatures, _timestamp, _proposer_addr):
    t = time.time()
    print(bcolors.OKCYAN, "BLOCK ", LATEST_BLOCK, bcolors.ENDC)
    result = {}
    all_threads = [
        threading.Thread(target=MyThread0, args=[result, LATEST_BLOCK]),
        threading.Thread(target=MyThread1, args=[result, "inflation_rate"]),
        threading.Thread(target=MyThread2, args=[result, "percent_staked"]),
        threading.Thread(target=MyThread3, args=[result, "total_block_fees", LATEST_BLOCK]),
        threading.Thread(target=MyThread4, args=[result, "proposer", _proposer_addr]),
        threading.Thread(target=MyThread4, args=[result, "timestamp", _timestamp]),
        threading.Thread(target=MyThread5, args=[result, "sign_ratio", _block_signatures]),
        threading.Thread(target=MyThread6, args=[result, "atom_staked_v"]),
        threading.Thread(target=MyThread7, args=[result, "total_supply"]),
        threading.Thread(target=MyThread9, args=[result])
    ]

    for val_ind, validator in enumerate(VALIDATOR_ADDRESS):
        all_threads.append(threading.Thread(target=MyThread10, args=[result, validator, val_ind]))

    for thread in all_threads:
        thread.start()
    for thread in all_threads:
        thread.join()

    dir_path = base_dir_path + f'model_data/Input Files/{datafile_name}.csv'
    print(bcolors.OKGREEN, "result for block ", LATEST_BLOCK," : ", bcolors.ENDC, result)
    try:
        df = pd.read_csv(dir_path)
    except:
        df = pd.DataFrame()
    df_ls = df.to_dict('records')
    df_ls.append(result)
    pd.DataFrame(df_ls).to_csv(dir_path, index=False)

    print(bcolors.OKCYAN, "Time taken for block : ", LATEST_BLOCK, ": ", time.time() - t, bcolors.ENDC)
    print()

Loaded libraries...
{'Block_length_target': 7.23302752293578, 'Goal_Bonded': 0.6666, 'Inflation_Rate_Change': 0.13, 'Min_Inflation_Rate': 0.07, 'Max_Inflation_Rate': 0.2, 'Min_Signatures': 0.6666, 'Blocks_per_year': 4360000, 'n_validators': 175, 'val_0_name': 'Coinbase', 'val_0_address': 'cosmosvaloper1c4k24jzduc365kywrsvf5ujz4ya6mwympnc4en', 'val_0_commission_pct': 20.0, 'val_1_name': 'Twinstake', 'val_1_address': 'cosmosvaloper1svwt2mr4x2mx0hcmty0mxsa4rmlfau4lwx2l69', 'val_1_commission_pct': 8.0, 'val_2_name': 'Figment', 'val_2_address': 'cosmosvaloper1hjct6q7npsspsg3dgvzk3sdf89spmlpfdn6m9d', 'val_2_commission_pct': 9.0}


### Running the DE code (part 1)

In [ ]:
past_block_num = 0
new_block = 0
num_signatures = 0
time_stamp = 0
got_blocks=0
threads_to_stop = []
while(got_blocks < N_BLOCKS_TO_GET):
    print("beginning thread ", got_blocks)
    while past_block_num == new_block:
        resp = requests.get(RPC_URL_PRIVATE+'/cosmos/base/tendermint/v1beta1/blocks/latest', headers=headers).json() # gets latest block number
        new_block = int(resp['block']['header']['height'])
        print(new_block)
        block_signatures=resp['block']['last_commit']['signatures']
        time_stamp = resp['block']['header']['time']
        proposer_addr = resp['block']['header']['proposer_address']
        time.sleep(1.5)

    new_block_thread = threading.Thread(target=get_all_block_data, args = [new_block, block_signatures, time_stamp, proposer_addr])
    print("started thread ", got_blocks)
    new_block_thread.start()
    print("ended thread", got_blocks)
    threads_to_stop.append(new_block_thread)

    past_block_num = new_block
    got_blocks+=1

print("End of Data Collection")


for i in threads_to_stop:
    i.join()

beginning thread  0
12957775
started thread  0
 BLOCK ended thread 12957775 
 0
beginning thread  1
1.2636184692382812e-05 s for thread 0
2.1457672119140625e-06 s for thread 3
2.1457672119140625e-06 s for thread 41.6689300537109375e-06 s for thread 4

9.512901306152344e-05 s for thread 5
1.5020370483398438e-05 s for thread 6
8.106231689453125e-06 s for thread 9
0.20219850540161133 s for thread 1
0.2093663215637207 s for thread7
0.21631431579589844 s for thread 10
0.22044682502746582 s for thread 10
0.22288727760314941 s for thread 10
0.45078277587890625 s for thread 2
 result for block  12957775  :   {'block_num': 12957775, 'Block_length_target': 7.23302752293578, 'Goal_Bonded': 0.6666, 'Inflation_Rate_Change': 0.13, 'Min_Inflation_Rate': 0.07, 'Max_Inflation_Rate': 0.2, 'Min_Signatures': 0.6666, 'Blocks_per_year': 4360000, 'n_validators': 175, 'val_0_name': 'Coinbase', 'val_0_address': 'cosmosvaloper1c4k24jzduc365kywrsvf5ujz4ya6mwympnc4en', 'val_0_commission_pct': 20.0, 'val_1_name': 

### Getting block fees (part 2)

In [ ]:
print("Getting block fees for previous blocks")
dir_path = base_dir_path+f'model_data/Input Files/{datafile_name}.csv'
df= pd.read_csv(dir_path)

for i in tqdm(range(len(df))):
    block_num = df.iloc[i]['block_num']
    row = df.iloc[i] 
    row['total_block_fees'] = get_total_fees(block_num)
    df.iloc[i] = row
    df.to_csv(dir_path, index=False)
    time.sleep(5)

Getting block fees for previous blocks


  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
100%|██████████| 30/30 [04:00<00:00,  8.03s/it]


### Post Processing

In [ ]:
# 
# Post processing
#
dir_path = base_dir_path+f'model_data/Input Files/{datafile_name}.csv'
processed_df = post_process_data(df, len(VALIDATOR_ADDRESS))
processed_df.to_csv(dir_path, index=False)

0
Rows appended:  0
